In [1]:
# Remove warning messages
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import random

# navigation and read files
import glob
import os

# Read HDF5 binary data format: convenient for large data storage
import h5py

# Read and display images
import matplotlib.pyplot as plt
#import imageio # Imageio is a Python library that provides an easy interface to read and write a wide range of image data,
#import cv2
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
#from tensorflow import keras

# DL dependancies
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, LeakyReLU,GlobalMaxPooling2D , BatchNormalization
from tensorflow.keras.metrics import top_k_categorical_accuracy,categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping


Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
X_train= pd.read_csv('/Users/fatimahgh/desktop/Arabic_Alphabet/X_train.csv')
X_test= pd.read_csv('/Users/fatimahgh/desktop/Arabic_Alphabet/X_test.csv')
y_train= pd.read_csv('/Users/fatimahgh/desktop/Arabic_Alphabet/y_train.csv')
y_test= pd.read_csv('/Users/fatimahgh/desktop/Arabic_Alphabet/y_test.csv')

In [4]:
print(X_train.shape) # x_train is the training data set.
print(y_train.shape) # is the set of labels to all the data in x_train.
print("+++++++++++++++++++++++++")
print(X_test.shape) # is the test data set.
print(y_test.shape) # is the set of labels to all the data in x_test

(37933, 1024)
(37933, 1)
+++++++++++++++++++++++++
(9501, 1024)
(9501, 1)


In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
y_test.columns

In [ ]:
y_train.iloc[37932]

In [ ]:
y_train.value_counts()

In [5]:
#convert the values to Float, 
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
y_train=y_train.astype('int32')-1
y_test=y_test.astype('int32')-1

# Then normalize values (From 0 to 1)
X_train/=255
X_test/=255

In [8]:
#preparing the data for CNN
#reshape the input to "1 chanal for binary images
X_train=X_train.values.reshape(X_train.shape[0],32,32,1)
X_test=X_test.values.reshape(X_test.shape[0],32,32,1)

In [9]:
from tensorflow.keras.utils import to_categorical
y_test = to_categorical(y_test,29)
y_train = to_categorical(y_train,29)

In [10]:
print(X_train.shape) # x_train is the training data set.
print(y_train.shape) # is the set of labels to all the data in x_train.
print("+++++++++++++++++++++++++")
print(X_test.shape) # is the test data set.
print(y_test.shape)

(37933, 32, 32, 1)
(37933, 29)
+++++++++++++++++++++++++
(9501, 32, 32, 1)
(9501, 29)


In [25]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense

def create_model(optimizer='adam', kernel_initializer='uniform', activation='relu'):
  # create model
  model = Sequential()
    
  model.add(Conv2D(filters=16, kernel_size=3, padding='same', strides =(1,1) ,input_shape=(32, 32, 1), kernel_initializer=kernel_initializer, activation=activation))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=2))
  
  model.add(Conv2D(filters=32, kernel_size=3, padding='same' , strides =(1,1), kernel_initializer=kernel_initializer, activation=activation))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=2))
  
  model.add(Conv2D(filters=128, kernel_size=3, padding='same', strides =(1,1), kernel_initializer=kernel_initializer, activation=activation))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=2))

  model.add(Conv2D(filters=256, kernel_size=3, padding='same', strides =(1,1), kernel_initializer=kernel_initializer, activation=activation))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=2))

  # flatten output of conv
  model.add(Flatten())
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.5))
#   model.add(BatchNormalization())


  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.5))
#   model.add(BatchNormalization())


  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
#   model.add(BatchNormalization())
  
  #Fully connected final layer 
  model.add(Dense(29, activation='softmax'))

  # Compile model
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)
  return model

In [26]:
model = create_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 32, 32, 16)        160       
_________________________________________________________________
batch_normalization_23 (Batc (None, 32, 32, 16)        64        
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 32)        4640      
_________________________________________________________________
batch_normalization_24 (Batc (None, 16, 16, 32)        128       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 8, 8, 128)        

In [29]:
# Training the model for # epochs
history = model.fit(X_train,y_train,epochs=50,verbose=1)

Epoch 1/50
1186/1186 [==============================] - 25s 21ms/step - loss: 0.2345 - accuracy: 0.9365
Epoch 2/50
1186/1186 [==============================] - 24s 21ms/step - loss: 0.2411 - accuracy: 0.9370
Epoch 3/50
1186/1186 [==============================] - 24s 21ms/step - loss: 0.2571 - accuracy: 0.9326
Epoch 4/50
1186/1186 [==============================] - 25s 21ms/step - loss: 0.2371 - accuracy: 0.9367
Epoch 5/50
1186/1186 [==============================] - 24s 21ms/step - loss: 0.2249 - accuracy: 0.9402
Epoch 6/50
1186/1186 [==============================] - 25s 21ms/step - loss: 0.2212 - accuracy: 0.9414
Epoch 7/50
1186/1186 [==============================] - 25s 21ms/step - loss: 0.2272 - accuracy: 0.9412
Epoch 8/50
1186/1186 [==============================] - 25s 21ms/step - loss: 0.2345 - accuracy: 0.9393
Epoch 9/50
1186/1186 [==============================] - 25s 21ms/step - loss: 0.2098 - accuracy: 0.9437
Epoch 10/50
1186/1186 [==============================] - 25s 21m

In [30]:
# Final evaluation of the model
metrics = model.evaluate(X_test, y_test, verbose=1)
print("Test Accuracy: {} %".format(metrics[1]*100))
print("Test Loss: {}".format(metrics[0]))

297/297 [==============================] - 2s 7ms/step - loss: 0.9881 - accuracy: 0.8723
Test Accuracy: 87.23292350769043 %
Test Loss: 0.9880748987197876
